### Initialization

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = os.getenv("AZURE_OPENAI_API_VERSION")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_EMBEDDING_MODEL"] = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL")

# print(os.getenv("AZURE_OPENAI_ENDPOINT"))
# print(os.getenv("AZURE_OPENAI_API_KEY"))
print(os.getenv("AZURE_OPENAI_API_VERSION"))
print(os.getenv("AZURE_OPENAI_EMBEDDING_MODEL"))

# print(os.getenv("AZURE_COSMOSDB_NOSQL_ENDPOINT"))
# print(os.getenv("AZURE_COSMOSDB_NOSQL_KEY"))
print(os.getenv("AZURE_COSMOSDB_NOSQL_DATABASE"))
print(os.getenv("AZURE_COSMOSDB_NOSQL_CONTAINER"))

config_list = [
    {
        "model": os.getenv("AZURE_OPENAI_CHAT_MODEL"),
        "api_key": os.getenv("AZURE_OPENAI_API_KEY"),
        "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
        "api_type": "azure",
        "api_version": os.getenv("AZURE_OPENAI_API_VERSION")
    }
]

llm_config = {"config_list": config_list, "timeout": 60, "temperature": 0, "seed": 1234}
print("LLM models: ", [config_list[i]["model"]
      for i in range(len(config_list))])

2024-05-01-preview
text-embedding-ada-002
rag-db
documents
LLM models:  ['gpt-4o']


### Azure Cosmos DB RetrieveUserProxyAgent

In [2]:
import autogen, time
from typing import Callable, Dict, List, Literal, Optional, Union
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from azure.cosmos import CosmosClient
from openai import AzureOpenAI

class AzureCosmosDBRetrieveUserProxyAgent(RetrieveUserProxyAgent):
    def __init__(
        self,
        name="CosmosDBRetrieveUserProxyAgent",
        human_input_mode: Literal["ALWAYS", "NEVER", "TERMINATE"] = "ALWAYS",
        is_termination_msg: Optional[Callable[[Dict], bool]] = None,
        retrieve_config: Optional[Dict] = None,  # config for the retrieve agent
        **kwargs,
    ):        
        super().__init__(name, human_input_mode, is_termination_msg, retrieve_config, **kwargs)

        client = CosmosClient(
            self._retrieve_config.get('AZURE_COSMOSDB_NOSQL_ENDPOINT'), 
            self._retrieve_config.get('AZURE_COSMOSDB_NOSQL_KEY')
        )
        database = client.get_database_client(self._retrieve_config.get('AZURE_COSMOSDB_NOSQL_DATABASE'))
        self.container = database.get_container_client(self._retrieve_config.get('AZURE_COSMOSDB_NOSQL_CONTAINER'))
        
        self.openai_client = AzureOpenAI(
            api_key = self._retrieve_config.get("AZURE_OPENAI_API_KEY"),  
            api_version = self._retrieve_config.get("AZURE_OPENAI_API_VERSION"),  
            azure_endpoint = self._retrieve_config.get("AZURE_OPENAI_ENDPOINT")
        )

    def generate_embeddings(self, text):
        """
        Generates embeddings for a given text using the OpenAI API v1.x
        """
        return self.openai_client.embeddings.create(
            input = text,
            model= self._retrieve_config.get("AZURE_OPENAI_EMBEDDING_MODEL")
        ).data[0].embedding

    def query_vector_db(
            self,
            query_texts: List[str],
            n_results: int = 10,
            search_string: str = "",
            **kwargs,
        ) -> Dict[str, Union[List[str], List[List[str]]]]:
        """
        Retrieve documents from the vector database that match the user problem, using cosine similarity

        Args:
            query_texts (str): the problem to be solved.
            n_results (int): the number of results to be retrieved. Default is 10.
            search_string (str): only docs that contain an exact match of this string will be retrieved. Default is "".

        Returns:
            A list of Azure Cosmos DB json documents
            id: unique id of the document
            content: textual representation of the document
            content_vector: vector representation of the document (1536 dimensions if text-embedding-ada-002 or 384 dimensions with text-embedding-3-small)
        """
        results = []
        for query_text in query_texts: 

            # vectorize the query
            query_vector = self.generate_embeddings(query_text)

            # find similar documents in Azure Cosmos DB for NoSQL
            # assume that the following fields exist in Cosmos DB: id, content, content_vector
            query_text = f"""
                SELECT TOP {n_results} c.id, c.content, VectorDistance(c.content_vector, {query_vector}) as cosine
                FROM c 
                ORDER BY VectorDistance(c.content_vector, {query_vector})
            """
            start_time = time.time()          

            items = self.container.query_items(
                query=query_text,
                parameters=None,
                enable_cross_partition_query=True
            )   
            
            results.append([
                ({
                    "id": item["id"], 
                    "content": item["content"], 
                    "content_vector": query_vector
                }, item["cosine"]) for item in items
            ])
                        
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Elapsed time: {elapsed_time:.6f} seconds")

        return results

    
    def retrieve_docs(self, problem: str, n_results: int = 10, search_string: str = "", **kwargs) -> Dict[str, Union[List[str], List[List[str]]]]:
        """
        Entry point for the RetrieveUseerProxyAgent

        Args:
            query_texts (str): the problem to be solved.
            n_results (int): the number of results to be retrieved. Default is 10.
            search_string (str): only docs that contain an exact match of this string will be retrieved. Default is "".
        """
        results = self.query_vector_db(
            query_texts = [problem],
            n_results = n_results,
            search_string = search_string,
            **kwargs,
        )
        self._results = results

### Create agents

In [3]:
def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

COMPLETION_PROMPT = "Reply `TERMINATE` after completing the task."

assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant." + COMPLETION_PROMPT,
    llm_config=llm_config
)

# Instantiate the User Proxy Agent
cosmosdb_retrieveuserproxyagent  = AzureCosmosDBRetrieveUserProxyAgent(
    name="AzureCosmosDBRetrieveUserProxyAgent",
    human_input_mode="NEVER",
    system_message = "You're an agent that can connect to a vector database and retrieve documents using cosine similarity. " + COMPLETION_PROMPT,
    max_consecutive_auto_reply=5,
    is_termination_msg = termination_msg,
    retrieve_config={
        "task": "qa",
        "docs_path": None,
        "AZURE_OPENAI_ENDPOINT": os.getenv("AZURE_OPENAI_ENDPOINT"),
        "AZURE_OPENAI_API_KEY": os.getenv("AZURE_OPENAI_API_KEY"),
        "AZURE_OPENAI_API_VERSION": os.getenv("AZURE_OPENAI_API_VERSION"),
        "AZURE_OPENAI_EMBEDDING_MODEL": os.getenv("AZURE_OPENAI_EMBEDDING_MODEL"),
        "AZURE_COSMOSDB_NOSQL_ENDPOINT": os.getenv("AZURE_COSMOSDB_NOSQL_ENDPOINT"),
        "AZURE_COSMOSDB_NOSQL_KEY": os.getenv("AZURE_COSMOSDB_NOSQL_KEY"),
        "AZURE_COSMOSDB_NOSQL_DATABASE": os.getenv("AZURE_COSMOSDB_NOSQL_DATABASE"),
        "AZURE_COSMOSDB_NOSQL_CONTAINER": os.getenv("AZURE_COSMOSDB_NOSQL_CONTAINER")
    },
    code_execution_config = {
        "work_dir": "coding",
        "use_docker": False
    }
)

c:\data\00_dev\MSFT-demos\autogen-cosmosdb-retrieve-user-proxy-agent\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
# Start logging
logging_session_id = autogen.runtime_logging.start(config={"dbname": "logs.db"})
print("Started Logging session ID: " + str(logging_session_id))

qa_problem = "Compare and contrast the various types of vector indexes in Azure Cosmos DB. Display the results in an easy-to-read table"

cosmosdb_retrieveuserproxyagent.initiate_chat(
    assistant, 
    message=cosmosdb_retrieveuserproxyagent.message_generator, 
    problem=qa_problem
)

# Stop logging
autogen.runtime_logging.stop()

Started Logging session ID: d979e286-453a-43fe-9ef1-a0aa13cebb75
Elapsed time: 0.689391 seconds
Adding content of doc 158a4e43-75d3-4016-8c7b-0867ae8d45d2 to context.
Adding content of doc 6dc15d4a-357a-4781-8878-30f7f199eada to context.
Adding content of doc f36efb44-58d0-48d0-be4f-188e1f40ef5c to context.
Adding content of doc ef7eada7-92b8-4a53-aea7-ad2f39ab15e4 to context.
Adding content of doc 10f41952-402f-421a-8049-9040ac9eeb7c to context.
Adding content of doc 765a6e5f-5df5-40da-9549-6d4518355a5f to context.
Adding content of doc d5e73298-6347-4f92-8a58-e2d82b928e78 to context.
Adding content of doc e3f9e863-2e39-472c-b641-2bd7596155fb to context.
Adding content of doc cbbaf89e-9150-4b4c-b7d6-8b4c10c1729e to context.
Adding content of doc d9f81494-bffc-405a-833e-bac67746c5ce to context.
Adding content of doc afe4614d-1d25-4503-8164-cb4a23cc09a9 to context.
Adding content of doc 6c2fe14c-3aad-42b9-b907-e2da6886f8f1 to context.
Adding content of doc 03abc57f-0b17-42f2-83e2-e8ad21